In [1]:
import pandas as pd
from pymongo import MongoClient
from datetime import timedelta
import numpy as np

def tweakcols(df):
    """Tweak columns to match required schema."""
    return (
        df.drop(columns=["Year", "Month", "Geo by City/ Area/ Town"])
        .dropna(subset="Date")
        .sort_values(by="Date", ascending=True)
        .assign(
            geography_breakdown=df["Geo by State"]
            .str.replace("Kuala Lumpur and Selangor", "Kuala Lumpur, Selangor")
            .str.replace("KL", "Kuala Lumpur"),
            Campaign=df.Campaign.str.replace("'", "")
            .str.replace("*", "")
            .str.replace("_", " "),
            Reach=df.Reach.replace("-", "")
            .replace("", np.nan)
            .astype("float")
            .fillna(0)
            .astype("int"),
            Impressions=df.Impressions.replace("-", "")
            .replace("", np.nan)
            .astype("float")
            .fillna(0)
            .astype("int"),
            Product=df.Product.fillna("All Products"),
            brand="Time",
            geography="Malaysia",
            currency_code="MYR",
            Date=df.Date.apply(lambda x:x-timedelta(1))
        )
    )


def lower_case_columns(df):
    """Rename columns to lower case according to schema."""
    new_names = df.columns.str.lower().tolist()
    df.columns = new_names
    return df


def rename_columns(df):
    """Rename columns according to schema."""
    return df.rename(
        columns={
            "channel": "media_channel",
            "funnel stage": "funnel_stage",
            "spends (rm)": "spend",
            "buy type": "buy_type",
        }
    )

def main(df):
    """Transform columns and data from PHD"""    
    columns = ['date', 'brand', 'category', 'product', 'media_channel',
            'geography', 'geography_breakdown', 'format', 'publisher',
            'campaign', 'funnel_stage', 'spend', 'reach', 'impressions', 'currency_code']

    cleaned_df = rename_columns(lower_case_columns(tweakcols(df)))
    return cleaned_df[columns]




path = "C:\\Users\\izzaz\\Documents\\1 Projects\\T - Onboarding of Mutinex MMM\\Data Provisioning\\TIME Campaign Weekly Breakdown - Client - R0 - 30 Sept.xlsx"

df0 = pd.read_excel(path)

df = main(df0)

C:\Users\izzaz\AppData\Local\Temp\ipykernel_19100\442122462.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace("", np.nan)
C:\Users\izzaz\AppData\Local\Temp\ipykernel_19100\442122462.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace("", np.nan)


In [2]:
df

,date,brand,category,product,media_channel,geography,geography_breakdown,format,publisher,campaign,funnel_stage,spend,reach,impressions,currency_code
15,2023-02-06,Time,FTTH,All Products,Radio,Malaysia,Nationwide,Radio Ad,REDI 98.8FM,CNY 2023,Awareness,26600.00,0,0,MYR
16,2023-02-13,Time,FTTH,All Products,Radio,Malaysia,Nationwide,Radio Ad,REDI 98.8FM,CNY 2023,Awareness,26600.00,0,0,MYR
17,2023-02-20,Time,FTTH,All Products,Radio,Malaysia,Nationwide,Radio Ad,REDI 98.8FM,CNY 2023,Awareness,26600.00,0,0,MYR
18,2023-02-27,Time,FTTH,All Products,Radio,Malaysia,Nationwide,Radio Ad,REDI 98.8FM,CNY 2023,Awareness,26600.00,0,0,MYR
19,2023-03-06,Time,FTTH,500mbps,Digital,Malaysia,"Johor, Kuala Lumpur, Penang, Selangor",Standard Native (Image),Yahoo,500Mbps at only RM99 for the first 6 months,Awareness,7816.42,0,4428752,MYR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,2024-05-06,Time,FTTH,All Products,Radio,Malaysia,Nationwide,Radio Ad,ERA FM,Network Superiority,Awareness,38310.00,0,0,MYR
60,2024-05-06,Time,FTTH,All Products,Digital,Malaysia,-,Social Media Posting,"Newswav, Siakap Keli Press",Time B2C Raya2024,Awareness,10000.00,286874,0,MYR
93,2024-05-13,Time,FTTH,All Products,Radio,Malaysia,Nationwide,Radio Ad,ERA FM,Network Superiority,Awareness,38310.00,0,0,MYR
94,2024-05-20,Time,FTTH,All Products,Radio,Malaysia,Nationwide,Radio Ad,ERA FM,Network Superiority,Awareness,38310.00,0,0,MYR


In [4]:
df.to_csv('phd_data_cleaned.csv', index=False)